# H1: Velocity + Временные признаки

**H1 (Product):** Скорость операций и cross-border транзакции повышают риск → шаг-up аутентификация при превышении порогов.

In [2]:
import sys
from pathlib import Path
ROOT = Path().resolve()
if not (ROOT/'src').exists(): ROOT = ROOT.parent
sys.path.insert(0, str(ROOT))
print('Project root:', ROOT)


Project root: /Users/gumerovbr/Documents/GitHub/data_analysis_itmo_2025


In [4]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.pipeline import Pipeline

from src.data import load_transactions, load_fx
from src.currency import convert_to_usd
from src.features import unpack_last_hour_activity, add_basic_time_features, customer_velocity, clip_and_fill
from src.validation import compute_time_cutoff, split_by_cutoff
from src.pipeline import build_preprocessor, build_logreg
from src.eval import eval_pack
from src.validation import bootstrap_pr_auc

# 1) Загрузка и подготовка
DATA = Path('../data')
TX = DATA/'transaction_fraud_data.parquet'
FX = DATA/'historical_currency_exchange.parquet'

df = load_transactions(TX)
fx = load_fx(FX)

df = convert_to_usd(df, fx)
df = unpack_last_hour_activity(df)
df = add_basic_time_features(df)

# ВАЖНО: сначала клиппинг/заполнение (убираем хвосты/пропуски ДО обучения)
df = clip_and_fill(df)

# 2) Кандидат: velocity (и тоже клиппинг/заполнение на тех же строках)
df_v = customer_velocity(df)
df_v = clip_and_fill(df_v)

# 3) Единый time split (один cutoff — два сплита)
cutoff = compute_time_cutoff(df, ts_col='timestamp', test_size=0.2)

train, test   = split_by_cutoff(df, cutoff, ts_col='timestamp')
train_v, test_v = split_by_cutoff(df_v, cutoff, ts_col='timestamp')

# Контроль, что разбиения совпали по индексам
assert train.index.equals(train_v.index)
assert test.index.equals(test_v.index)

# 4) Обучение: baseline
y_tr, y_te = train['is_fraud'].astype(int), test['is_fraud'].astype(int)
X_tr, X_te = train.drop(columns=['is_fraud']), test.drop(columns=['is_fraud'])

pipe_base = Pipeline([
    ('prep', build_preprocessor(X_tr)),
    ('clf', build_logreg()),
]).fit(X_tr, y_tr)

p_base = pipe_base.predict_proba(X_te)[:, 1]

# 5) Обучение: + velocity
y_tr_v, y_te_v = train_v['is_fraud'].astype(int), test_v['is_fraud'].astype(int)
X_tr_v, X_te_v = train_v.drop(columns=['is_fraud']), test_v.drop(columns=['is_fraud'])

pipe_v = Pipeline([
    ('prep', build_preprocessor(X_tr_v)),
    ('clf', build_logreg()),
]).fit(X_tr_v, y_tr_v)

p_v = pipe_v.predict_proba(X_te_v)[:, 1]

# 6) Метрики (одинаковая тестовая выборка!)
base_metrics = eval_pack(y_te, p_base)
cand_metrics = eval_pack(y_te_v, p_v)

# 7) Bootstrap дельты AP с согласованным y_true
delta, lo, hi = bootstrap_pr_auc(y_te.to_numpy(), p_base, p_v, B=1000, seed=42)

base_metrics, cand_metrics, {'delta_pr': delta, '95%CI': (lo, hi)}

/Users/gumerovbr/Documents/GitHub/data_analysis_itmo_2025/.venv/lib/python3.9/site-packages/sklearn/linear_model/_linear_loss.py:209: RuntimeWarning: divide by zero encountered in matmul
  norm2_w = weights @ weights if weights.ndim == 1 else squared_norm(weights)
/Users/gumerovbr/Documents/GitHub/data_analysis_itmo_2025/.venv/lib/python3.9/site-packages/sklearn/linear_model/_linear_loss.py:209: RuntimeWarning: overflow encountered in matmul
  norm2_w = weights @ weights if weights.ndim == 1 else squared_norm(weights)
/Users/gumerovbr/Documents/GitHub/data_analysis_itmo_2025/.venv/lib/python3.9/site-packages/sklearn/linear_model/_linear_loss.py:209: RuntimeWarning: invalid value encountered in matmul
  norm2_w = weights @ weights if weights.ndim == 1 else squared_norm(weights)
/Users/gumerovbr/Documents/GitHub/data_analysis_itmo_2025/.venv/lib/python3.9/site-packages/sklearn/linear_model/_linear_loss.py:209: RuntimeWarning: divide by zero encountered in matmul
  norm2_w = weights @ wei

({'roc_auc': 0.971676410870483,
  'pr_auc': 0.9249518117159337,
  'threshold_at_precision': 0.9,
  'thr_value': 0.3122758056285582,
  'recall_at_precision': 0.7514503899187782,
  'precision_achieved': 0.9000028278600775},
 {'roc_auc': 0.971681985928168,
  'pr_auc': 0.9249480817960043,
  'threshold_at_precision': 0.9,
  'thr_value': 0.22828390395114512,
  'recall_at_precision': 0.7518113008985672,
  'precision_achieved': 0.9000024227153794},
 {'delta_pr': -2.6066334293508573e-06,
  '95%CI': (-3.159779047487432e-05, 2.5990901927425147e-05)})

Отлично. Результаты выглядят стабильными и «здоровыми» с точки зрения численной устойчивости:
* ROC AUC: 0.971676 → 0.971682 (≈ +0.000006)
* PR AUC: 0.924952 → 0.924948 (≈ −0.000004)
* Recall @ P≥0.90: 0.75145 → 0.75181 (незначительный рост)
* Порог для P≈0.90: 0.312 → 0.228 (вариант достигает той же точности при меньшем пороге)
* ΔPR (bootstrap, B=1000): −2.6e−06, 95% CI [−3.16e−05; +2.60e−05] → статзначимого прироста AP нет

Вывод по H1 сейчас

В текущей реализации velocity-признаки не дают статистически значимого прироста AP на общей выборке. При этом наблюдается слабый сдвиг в сторону более «щедрой» отсечки (ниже порог при той же Precision) и микроприращение recall @ P≥0.90. Это может быть полезно локально по сегментам (например, cross-border), но не видно эффекта «в среднем по больнице».

Ниже — короткий план, как оперативно «дожать» гипотезу и принять продуктовое решение.

⸻

Что сделать дальше (приоритетно)
	1.	Проверить uplift по целевым сегментам (там, где H1 задумана):

* is_outside_home_country == 1 (cross-border),
* высокорисковые категории мерчантов,
* ночные часы/выходные.

Если uplift концентрируется в этих зонах — включаем step-up только в них, а не глобально.
	2.	Посмотреть вклад именно velocity-фичей в модели:

* знак/величина коэффициентов (odds ratio),
* монотония эффектов (рост риска при росте скорости).

	3.	Добавить интеракции «velocity × cross-border / high-risk / is_card_present»:

* именно такие взаимодействия чаще всего «раскрывают» H1.

	4.	Проверить устойчивость во времени (rolling backtest, 3–5 временных фолдов) — чтобы убедиться, что эффект не артефакт единственного среза.
	5.	Пересчитать бизнес-метрики на уровне порога P≥0.90:

* прирост True Positive, рост нагрузки на step-up (FP) и итоговый net benefit.

In [ ]:
# 1) Коэффициенты/odds ratio, выделить velocity

import numpy as np
import pandas as pd

def coef_table(pipe):
    prep = pipe.named_steps['prep']
    clf  = pipe.named_steps['clf']
    feat = prep.get_feature_names_out()
    coefs = clf.coef_.ravel()
    dfc = pd.DataFrame({'feature': feat, 'coef': coefs})
    dfc['odds_ratio'] = np.exp(dfc['coef'])
    return dfc.sort_values(dfc.columns[dfc.columns.get_loc('coef')], key=lambda s: s.abs(), ascending=False)

coef_base = coef_table(pipe_base)
coef_v    = coef_table(pipe_v)

# velocity-фичи
mask_vel = coef_v['feature'].str.contains(r'(time_since_prev_s|cust_tx_count_1h|cust_tx_count_6h|cust_tx_count_24h)')
coef_v[mask_vel].sort_values('coef', ascending=False)

/var/folders/xc/r2jvff2n1_s_9sgktp8v3q300000gn/T/ipykernel_8332/2815852813.py:19: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask_vel = coef_v['feature'].str.contains(r'(time_since_prev_s|cust_tx_count_1h|cust_tx_count_6h|cust_tx_count_24h)')


,feature,coef,odds_ratio
15,num__cust_tx_count_24h,0.051656,1.053013
12,num__time_since_prev_s,0.037748,1.038469
14,num__cust_tx_count_6h,-0.010339,0.989714
13,num__cust_tx_count_1h,-0.045046,0.955954


In [ ]:
# 2) Uplift по сегментам (PR AUC и Recall@P≥0.90)

from sklearn.metrics import average_precision_score, precision_recall_curve

def recall_at_precision(y, p, target=0.90):
    prec, rec, thr = precision_recall_curve(y, p)
    m = prec[:-1] >= target
    return float(rec[:-1][m].max()) if m.any() else None

def segment_report(mask, name):
    y = y_te_v[mask]
    b = p_base[mask]
    v = p_v[mask]
    rep = {
        'seg': name,
        'n': int(mask.sum()),
        'ap_base': float(average_precision_score(y, b)),
        'ap_var':  float(average_precision_score(y, v)),
        'rec@P0.90_base': recall_at_precision(y, b, 0.90),
        'rec@P0.90_var':  recall_at_precision(y, v, 0.90),
    }
    return rep

reports = []
# 2.1 Cross-border
reports.append(segment_report(test_v['is_outside_home_country'] == 1, 'cross_border'))
# 2.2 Ночь (например, 0-6)
reports.append(segment_report((test_v['tx_hour']>=0) & (test_v['tx_hour']<=6), 'night'))
# 2.3 High-risk vendor
if 'is_high_risk_vendor' in test_v.columns:
    reports.append(segment_report(test_v['is_high_risk_vendor']==1, 'high_risk_vendor'))

pd.DataFrame(reports)

,seg,n,ap_base,ap_var,rec@P0.90_base,rec@P0.90_var
0,cross_border,479994,0.942141,0.942171,0.773749,0.774714
1,night,326051,0.967413,0.967243,0.913099,0.912516
2,high_risk_vendor,374684,0.919979,0.920010,0.735781,0.736092


In [9]:
# 3) Интеракции (быстрый прототип)

# Добавьте к df_v взаимодействия перед обучением:

df_vi = df_v.copy()
df_vi['vel1_x_xborder'] = df_vi['cust_tx_count_1h'] * df_vi['is_outside_home_country']
df_vi['vel6_x_xborder'] = df_vi['cust_tx_count_6h'] * df_vi['is_outside_home_country']
df_vi['vel24_x_xborder']= df_vi['cust_tx_count_24h'] * df_vi['is_outside_home_country']

# Дальше тот же fixed cutoff split и обучение, как у df_v
train_vi, test_vi = split_by_cutoff(df_vi, cutoff, ts_col='timestamp')
y_tr_vi, y_te_vi = train_vi['is_fraud'].astype(int), test_vi['is_fraud'].astype(int)
X_tr_vi, X_te_vi = train_vi.drop(columns=['is_fraud']), test_vi.drop(columns=['is_fraud'])

pipe_vi = Pipeline([('prep', build_preprocessor(X_tr_vi)), ('clf', build_logreg())]).fit(X_tr_vi, y_tr_vi)
p_vi = pipe_vi.predict_proba(X_te_vi)[:,1]

from src.eval import eval_pack
eval_pack(y_te_vi, p_vi)

/Users/gumerovbr/Documents/GitHub/data_analysis_itmo_2025/.venv/lib/python3.9/site-packages/sklearn/linear_model/_linear_loss.py:209: RuntimeWarning: divide by zero encountered in matmul
  norm2_w = weights @ weights if weights.ndim == 1 else squared_norm(weights)
/Users/gumerovbr/Documents/GitHub/data_analysis_itmo_2025/.venv/lib/python3.9/site-packages/sklearn/linear_model/_linear_loss.py:209: RuntimeWarning: overflow encountered in matmul
  norm2_w = weights @ weights if weights.ndim == 1 else squared_norm(weights)
/Users/gumerovbr/Documents/GitHub/data_analysis_itmo_2025/.venv/lib/python3.9/site-packages/sklearn/linear_model/_linear_loss.py:209: RuntimeWarning: invalid value encountered in matmul
  norm2_w = weights @ weights if weights.ndim == 1 else squared_norm(weights)


{'roc_auc': 0.9717569885306397,
 'pr_auc': 0.9253000437697738,
 'threshold_at_precision': 0.9,
 'thr_value': 0.5556421146023413,
 'recall_at_precision': 0.7528839148385008,
 'precision_achieved': 0.9}

In [ ]:
# 4) Rolling backtest (устойчивость во времени)

def rolling_splits(df, ts_col='timestamp', k=3):
    df = df.sort_values(ts_col).reset_index(drop=True)
    n = len(df)
    fold = []
    for i in range(1, k+1):
        cut = int(n*(1 - i/(k+1)))
        train = df.iloc[:cut]
        test  = df.iloc[cut:]
        fold.append((train, test))
    return fold

def evaluate_pipe(train, test):
    y_tr, y_te = train['is_fraud'].astype(int), test['is_fraud'].astype(int)
    X_tr, X_te = train.drop(columns=['is_fraud']), test.drop(columns=['is_fraud'])
    pipe = Pipeline([('prep', build_preprocessor(X_tr)), ('clf', build_logreg())]).fit(X_tr, y_tr)
    p = pipe.predict_proba(X_te)[:,1]
    return eval_pack(y_te, p)

for name, data in [('baseline', df), ('velocity', df_v)]:
    results = []
    for tr, te in rolling_splits(data, 'timestamp', k=3):
        results.append(evaluate_pipe(tr, te))
    print(name, results)

In [ ]:
# 5) Калибровка и Brier score (на случай продуктового порога)

from sklearn.calibration import calibration_curve
from sklearn.metrics import brier_score_loss

brier_base = brier_score_loss(y_te, p_base)
brier_var  = brier_score_loss(y_te_v, p_v)
brier_base, brier_var

Решение «по продукту» (что зафиксировать)
* Если uplift появляется в целевых сегментах (например, cross-border) — оставляем velocity-признаки и включаем step-up только в этих сегментах. На общей популяции — эффект ≈0.
* Если uplift нет даже по сегментам — фиксируем, что H1 не подтверждена в текущем виде; пробуем интеракции (см. 3) и/или бининг velocity в монотоничные сплайны/квантили.
* В любом случае: у вас теперь реплицируемый эксперимент (единый cutoff, стабильные пайплайны). Сохраните артефакты (версия данных, cutoff, seed, метрики) в отчёт, чтобы закрыть гипотезу формально.